In [1]:
from experiments import FairletExperiment
from helper.data_loader import data_loader

In [6]:
#set constants
#dataset_names = ["adult", "bank"]
dataset_names = ["adult"]
clustering_method = "k-centers"
num_experiments = 5
starting_randomState = 42
starting_num_clusters = 3
max_num_clusters = 20

In [3]:
loader = data_loader('./config/dataset_config.json')
config = loader.get_config()

In [4]:
fairletDict = {}


for dataset_name in dataset_names:
    X,_ = loader.load_dataset(dataset_name)
    prepared_dataset = loader.prepare_dataset(X, dataset_name, complexity="extended")
    normalized_dataset = loader.normalize_data(prepared_dataset, dataset_name)

    validRandomStates = FairletExperiment.FindValidRandomStates(normalized_dataset, dataset_name, loader, num_experiments, starting_randomState)
    print(validRandomStates)
    fairletTuple = FairletExperiment.DoFairletExperiment(normalized_dataset, dataset_name, loader, validRandomStates, "fairlets", clustering_method , starting_num_clusters, max_num_clusters, "min")
    idString = "fairlet_" + dataset_name + "_" + clustering_method
    fairletDict[idString] = fairletTuple


[43, 44, 46, 47, 49]
cluster count: 3
cluster count: 4
cluster count: 5
cluster count: 6
cluster count: 7
cluster count: 8
cluster count: 9
cluster count: 10
cluster count: 11
cluster count: 12
cluster count: 13
cluster count: 14
cluster count: 15
cluster count: 16
cluster count: 17
cluster count: 18
cluster count: 19
cluster count: 20
cluster count: 3
cluster count: 4
cluster count: 5
cluster count: 6
cluster count: 7
cluster count: 8
cluster count: 9
cluster count: 10
cluster count: 11
cluster count: 12
cluster count: 13
cluster count: 14
cluster count: 15
cluster count: 16
cluster count: 17
cluster count: 18
cluster count: 19
cluster count: 20
cluster count: 3
cluster count: 4
cluster count: 5
cluster count: 6
cluster count: 7
cluster count: 8
cluster count: 9
cluster count: 10
cluster count: 11
cluster count: 12
cluster count: 13
cluster count: 14
cluster count: 15
cluster count: 16
cluster count: 17
cluster count: 18
cluster count: 19
cluster count: 20
cluster count: 3
cluster cou

AttributeError: 'tuple' object has no attribute 'append'